# TP2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Standard normal (from TP1)

def one_sample_uniform(increased_support_list):
    k = 0
    U = np.random.rand()
    sum_proba = 1 / len(increased_support_list)
    while (U > sum_proba):
        k += 1
        sum_proba += 1 / len(increased_support_list)

    return increased_support_list[k]

def sample_abs_std_normal(N):
    res = []

    f = lambda x : 2 / np.sqrt(2*np.pi) * np.exp(-x**2/2) * int(x>0)
    g = lambda x : np.exp(-x) * int(x>0)
    inv_cdf_g = lambda x : -np.log(1-x) * int(x>0 and x<1)
    c = np.sqrt(2 * np.e / np.pi)

    for _ in range(N):
        while True:
            U = np.random.rand()

            U_ = np.random.rand()
            Y = inv_cdf_g(U_)

            if U < f(Y) / (c * g(Y)):
                break
        res.append(Y)
    
    return res

def sample_std_normal(N):
    return [one_sample_uniform([-1,1]) * i for i in sample_abs_std_normal(N)]

def sample_normal(N, mu, sigma):
    return [z*sigma+mu for z in sample_std_normal(N)]

# f = lambda x, mu=2, sigma=3: 1/(np.sqrt(2*np.pi)*sigma) * np.exp(-((x-mu)/sigma)**2/2)

## Exercice 1

1.

$$\mathbb{E}[X] = -\frac{1}{3} + \frac{1}{2} = \frac{1}{6}$$
$$\mathbb{E}[Y] = -p_{-1} + p_{1}$$

\begin{equation*}
\begin{cases}
-p_{-1} + p_{1} = \frac{1}{6}\\
p_{-1} + p_{1} = \frac{1}{5}
\end{cases}

\Rightarrow

\begin{cases}
p_{1} = \frac{11}{60}\\
p_{-1} = \frac{1}{60}
\end{cases}
\end{equation*}

In [ ]:
# 3.

def discrete_sample(support_list, proba_list, N):
    c_list = [sum(proba_list[0:(i+1)]) for i in range(len(proba_list))]

    res = []

    for _ in range(N):
        k = 0
        U = np.random.rand()
        while (U > c_list[k] and k < len(proba_list)):
            k += 1
        res.append(support_list[k])

    return res

X_N_bar = [np.mean(discrete_sample([-1,0,1], [1/3,1/6,1/2], N)) for N in range(1,101)]
Y_N_bar = [np.mean(discrete_sample([-1,0,1], [1/60,4/5,11/60], N)) for N in range(1,101)]

plt.plot([i for i in range(1,101)], X_N_bar)
plt.plot([i for i in range(1,101)], Y_N_bar)

## Exercice 2

In [ ]:
# 2.

mu = 0
n = 30
x0 = 20
sigma = 0.9
a = 8
b = 36

N = 10000

X_k1 = lambda X_k: X_k * np.exp((mu-sigma**2/2)/n + (sigma*np.sqrt(1/n)*sample_std_normal(1)[0]))

X_list = []
for _ in range(N):
    X_list.append([x0])
    for _ in range(n):
        X_list[-1].append(X_k1(X_list[-1][-1]))
    plt.plot([i for i in range(n+1)], X_list[-1])

In [ ]:
h = lambda x: min(abs(x-a), abs(x-b))

mc_est = sum([ h(X_list[i][-1]) * np.prod([int(a <= X_k and X_k <= b) for X_k in X_list[i]]) for i in range(N) ]) / N
mc_est

In [ ]:
# 3.

alpha = 0.05
IC = (mc_est - norm.ppf(1-alpha/2)*sigma/np.sqrt(N), mc_est + norm.ppf(1-alpha/2)*sigma/np.sqrt(N))
IC

## Exercice 3

In [ ]:
# 2.

sigma = 2
c = 4.5
N = 10000

g = lambda x: x * int(x > c)

mc_est = sum(list(map(g, sample_normal(N, 0, sigma)))) / N
mc_est

Intervalle de confiance:

$$IC = \left[ \bar{X_n}-q_{1-\frac{\alpha}{2}}\frac{\hat{\sigma}}{\sqrt{n}}; \bar{X_n}+q_{1-\frac{\alpha}{2}}\frac{\hat{\sigma}}{\sqrt{n}} \right]$$

$$\hat{\sigma}^2 = \frac{1}{n-1} \sum_{k=1}^{n} (X_k - \bar{X_n})^2$$

In [ ]:
# 3.

N = 500

g_X_list = [list(map(g, sample_normal(n, 0, sigma))) for n in range(1,N+1)]
Y_bar_list = [sum(g_X_list[i])/(i+1) for i in range(len(g_X_list))]

sigma_hat = lambda X_list: np.sqrt(1/(len(X_list) - (1 if len(X_list)>1 else 0)) * sum([(x - np.mean(X_list))**2 for x in X_list]))

alpha = 0.05    # 95% confidence interval
IC = [
    (Y_bar_list[i] - norm.ppf(1-alpha/2)*sigma_hat(g_X_list[i])/np.sqrt(i+1),
    Y_bar_list[i] + norm.ppf(1-alpha/2)*sigma_hat(g_X_list[i])/np.sqrt(i+1)) 
    for i in range(len(Y_bar_list))
]

plt.plot([n for n in range(1,N+1)], Y_bar_list, linestyle='None', markersize=5, marker='.')
plt.plot([n for n in range(1,N+1)], [IC[i][0] for i in range(N)], linestyle='None', markersize=5, marker='.')
plt.plot([n for n in range(1,N+1)], [IC[i][1] for i in range(N)], linestyle='None', markersize=5, marker='.')

4.(a)

$$\mathbb{E}[g(X)] = \int g(x)h(x)dx = \int g(x) \frac{h(x)}{h_\mu(x)} h_\mu(x)dx = \mathbb{E}[\psi (Z^\mu)]$$

$$h(x) = \frac{1}{\sigma\sqrt{2\pi}} \exp \left( -\frac{x^2}{2\sigma^2} \right), h_\mu(x) = \frac{1}{\sigma\sqrt{2\pi}} \exp \left( -\frac{(x-\mu)^2}{2\sigma^2} \right)$$

$$\psi(z) = g(z)\frac{h(z)}{h_\mu(z)} = z\mathbb{1}_{\{z>c\}} \times \exp{\left( \frac{-2\mu z + \mu^2}{2\sigma^2} \right)}$$

4.(b)

$$Z^\mu = \mu + \sigma\xi$$

$$\mathbb{E}[\psi^2(\mu+\sigma\xi)] = \mathbb{E} \left[ g^2(\mu+\sigma\xi)\exp \left( \frac{-\mu(\mu+2\sigma\xi)}{\sigma^2} \right) \right] = \mathbb{E}[K(\mu,\xi)]$$

In [ ]:
# 4.(c)

K = lambda mu, xi: g(mu+sigma*xi)**2 * np.exp(-mu*(mu+2*sigma*xi)/(sigma**2))

mu_list = list(np.arange(0,7,0.01))
K_tilde_list = [
    sum(list(map(K, [mu]*N, sample_std_normal(N)))) / N
    for mu in mu_list
]
plt.plot(mu_list, K_tilde_list, marker='.', linestyle='None')

4.(d)

$$K'(\mu,\xi) = 2\exp \left( \frac{-\mu(\mu+2\sigma\xi)}{\sigma^2} \right) \left[ g'(\mu+\sigma\xi)g(\mu+\sigma\xi) - \frac{\mu+\sigma\xi}{\sigma^2} g^2(\mu+\sigma\xi) \right]$$

$$K''(\mu,\xi) = ...$$

## Exercice 4

In [ ]:
# 1.

T = 1
n = 50
sigma1 = 0.5
sigma2 = 0.8
r = 0.03
X0 = 50
Y0 = 60
K = 55

delta = T/n

Z_list = sample_std_normal(n)
Z_tilde_list = sample_std_normal(n)

# 1.(a)

X_list = [X0]
Y_list = [Y0]

rho = 0.5

for k in range(n):
    X_list.append(
        X_list[k] * ((1 + r*delta) + sigma1*np.sqrt(delta)*Z_list[k])
    )
    Y_list.append(
        Y_list[k] * ((1 + r*delta) + rho*sigma2*np.sqrt(delta)*Z_list[k] + sigma2*np.sqrt(1-rho**2)*np.sqrt(delta)*Z_tilde_list[k])
    )

plt.plot([i for i in range(n+1)], X_list)
plt.plot([i for i in range(n+1)], Y_list)

In [ ]:
# 1.(b)

N = 1000

X_tn = []
Y_tn = []

for _ in range(N):
    Z_list = sample_std_normal(n)
    Z_tilde_list = sample_std_normal(n)

    X_list = [X0]
    Y_list = [Y0]

    rho = 0.5

    for k in range(n):
        X_list.append(
            X_list[k] * ((1 + r*delta) + sigma1*np.sqrt(delta)*Z_list[k])
        )
        Y_list.append(
            Y_list[k] * ((1 + r*delta) + rho*sigma2*np.sqrt(delta)*Z_list[k] + sigma2*np.sqrt(1-rho**2)*np.sqrt(delta)*Z_tilde_list[k])
        )
    
    X_tn.append(X_list[-1])
    Y_tn.append(Y_list[-1])

est = np.exp(-r*T) * sum([
    max(0, 0.5*X_tn[i] + 0.5*Y_tn[i] - K)
    for i in range(N)
]) / N

est